## Loading Modules

In [1]:
import os
import numpy as np
import pandas as pd

from utils import *
from calibration import *

from synapticTrack.io import BeamDataIOManager
from synapticTrack.lattice.track_elements import *
from synapticTrack.lattice.track_lattice import Lattice
from synapticTrack.track.run_track import *

## Run Track Simulations

In [2]:
import time
t0 = time.time()

# Load LEBT lattice
elements_list, stms = load_LEBT_lattice()

# ouput files and directories
output_dir_list = ['run.wire_scanner1', 'run.allison_scanner', 'run.wire_scanner2', 'run.wire_scanner3', 'run.wire_scanner4', 'run.ends']

final_output_files = ['coord_wire_scanner1.out', 'coord_allison_scanner.out', 'coord_wire_scanner2.out', 'coord_wire_scanner3.out', 
                      'coord_wire_scanner4.out', 'coord_ends.out']

fh = [5.0, 5.0, 5.0, 5.0, 5.0]
fv = [5.0, 5.0, 5.0, 5.0, 5.0]

sim_centroid_x = np.zeros((4, 5, 3), dtype=float)
sim_centroid_y = np.zeros((4, 5, 3), dtype=float)
count = 0

for i, stm in enumerate(['stm0', 'stm1', 'stm2', 'stm3', 'stm4']):
    for j, dir_end, kick in zip([0, 1, 2], ['p', 'm', 'n'], [1.1, -1.1, 0.0]):
        for k, direction in enumerate(['x', 'y']):
            fhkick = [0.0, 0.0, 0.0, 0.0, 0.0]
            fvkick = [0.0, 0.0, 0.0, 0.0, 0.0]
            if direction == 'x':
                fhkick[i] = kick
            elif direction == 'y':
                fvkick[i] = kick

            final_output_dir = "stm." + str(i) + direction + dir_end
            os.makedirs(final_output_dir, exist_ok=True)

            set_stm_strengths(stms, fhkick, fvkick)
            #elements_list = construct_lattice_segments()

            # run track
            run_track_steps(elements_list, output_dir_list)
            copy_step_output_files(final_output_dir, output_dir_list, final_output_files)
            copy_track_output_files(final_output_dir, output_dir_list)

            # compute beam centroids and rms sizes
            centroid, rms = get_beam_data(final_output_dir, final_output_files)
            x_center = centroid['x'].apply(lambda x: float(f"{x: 11.8f}")).tolist()
            y_center = centroid['y'].apply(lambda x: float(f"{x: 11.8f}")).tolist()

            for l, ws in enumerate([0, 2, 3, 4]):
                if direction == 'x':
                    sim_centroid_x[l, i, j] = x_center[ws]
                elif direction == 'y':
                    sim_centroid_y[l, i, j] = y_center[ws]
            
            print (f"stm{i}", f"{kick: 4.1f}", direction, x_center, y_center)

np.save("sim_centroid_x.npy", sim_centroid_x)
np.save("sim_centroid_y.npy", sim_centroid_y)

t1 = time.time()
print (t1 - t0)

stm0  1.1 x [0.44827917, 0.85403108, -0.07786783, 0.52131927, 0.65830206, -0.05741116] [8.2e-07, -1.27e-06, 0.00057116, -0.0003598, -0.0002058, 5.28e-05]
stm0  1.1 y [0.04237972, -0.02638685, 0.09738859, -0.11740516, -0.23092948, -0.01096819] [0.40590053, 1.10889909, -1.03559181, 0.39090027, 1.20776584, -0.07714896]
stm0 -1.1 x [-0.36352059, -0.90680356, 0.27146385, -0.75418515, -1.11678377, 0.03552886] [8.2e-07, -4.9e-07, 0.00056813, -0.00035634, -0.00020182, 5.006e-05]
stm0 -1.1 y [0.04237972, -0.02638667, 0.09738849, -0.11741008, -0.23093369, -0.01096652] [-0.40589912, -1.10890137, 1.03673315, -0.39161712, -1.20817511, 0.07725225]
stm0  0.0 x [0.04237972, -0.02638572, 0.09633771, -0.11552018, -0.22775217, -0.01094204] [8.2e-07, -7.5e-07, 0.00057037, -0.00035851, -0.0002041, 5.167e-05]
stm0  0.0 y [0.04237972, -0.02638572, 0.09633771, -0.11552018, -0.22775217, -0.01094204] [8.2e-07, -7.5e-07, 0.00057037, -0.00035851, -0.0002041, 5.167e-05]
stm1  1.1 x [0.04237972, 1.18300829, -1.3420

## Calibrate Steering Magnet

In [3]:
# ---------------------------------------------------------------
# Load numpy arrays (shape = (4 WS, 5 steerers, 3 kicks))
# kicks index order assumed to be: [ +1.1 mrad , -1.1 mrad , 0.0 ]
# ---------------------------------------------------------------
exp_x = np.load("exp_centroid_x.npy")   # shape (4,5,3)
exp_y = np.load("exp_centroid_y.npy")
sim_x = np.load("sim_centroid_x.npy")
sim_y = np.load("sim_centroid_y.npy")

print("Loaded centroid arrays with shape:", exp_x.shape)

WS = 4
STEERERS = 5
KICK_PLUS  = 0
KICK_MINUS = 1
KICK_ZERO  = 2

KICK_MAG = 1.1  # mrad nominal

Loaded centroid arrays with shape: (4, 5, 3)


In [4]:
# ---------------------------------------------------------------
# Check data
# ---------------------------------------------------------------

check_data(WS, STEERERS, exp_x, sim_x, exp_y, sim_y)

stm0: ws0:  0.521  0.448 -0.149 -0.364  0.815  0.406  0.108 -0.406
stm0: ws1: -1.735 -0.078 -1.382  0.271 -0.601 -1.036  1.500  1.037
stm0: ws2:  2.349  0.521  1.352 -0.754  1.185  0.391  0.335 -0.392
stm0: ws3:  2.712  0.658  1.061 -1.117 -1.764  1.208 -4.023 -1.208

stm1: ws0:  0.000  0.042  0.000  0.042  0.000  0.000  0.000  0.000
stm1: ws1: -2.940 -1.342 -0.348  1.537 -0.420 -0.586  1.569  0.587
stm1: ws2:  2.973  0.969  0.823 -1.208  0.824 -0.294  0.711  0.294
stm1: ws3:  4.902  2.465 -0.493 -2.931 -1.405  1.438 -4.843 -1.439

stm2: ws0:  0.000  0.042  0.000  0.042  0.000  0.000  0.000  0.000
stm2: ws1: -1.841 -0.033 -1.542  0.227  1.828  1.617 -0.535 -1.616
stm2: ws2:  1.431 -0.825  2.424  0.592 -0.184 -1.250  1.534  1.249
stm2: ws3:  1.757 -0.879  2.700  0.422 -3.611 -0.950 -2.305  0.949

stm3: ws0:  0.000  0.042  0.000  0.042  0.000  0.000  0.000  0.000
stm3: ws1: -1.177  0.696 -2.202 -0.504  1.220  0.884 -0.393 -0.883
stm3: ws2:  1.590 -0.454  2.218  0.223  0.721 -0.092  0.956

In [5]:
# ---------------------------------------------------------------
# Compute Wire Scanner Offsets
# ---------------------------------------------------------------

ws_offset_x, ws_offset_y = compute_wire_scanner_offsets(WS, exp_x, sim_x, exp_y, sim_y, KICK_ZERO)

np.save("ws_offset_x.npy", ws_offset_x)
np.save("ws_offset_y.npy", ws_offset_y)

== Wire Scanner Offsets (simulation - experiment) ===
WS1: x_offset=-0.2666,   y_offset=-1.0617
WS2: x_offset=+1.0788,   y_offset=-0.3785
WS3: x_offset=-0.8454,   y_offset=-0.5858
WS4: x_offset=-0.2576,   y_offset=+2.7373


In [6]:
# ---------------------------------------------------------------
# Compute response matrices
# ---------------------------------------------------------------

from calibration import *
R_sim_x, R_exp_x = compute_response(WS, STEERERS, sim_x, exp_x, KICK_PLUS, KICK_MINUS, KICK_MAG)
R_sim_y, R_exp_y = compute_response(WS, STEERERS, sim_y, exp_y, KICK_PLUS, KICK_MINUS, KICK_MAG)

In [7]:
# ---------------------------------------------------------------
# Compute scale factors
# ---------------------------------------------------------------

scale_x = solve_scale_factors(STEERERS, R_sim_x, R_exp_x)
scale_y = solve_scale_factors(STEERERS, R_sim_y, R_exp_y)

np.save("stm_scale_x.npy", scale_x)
np.save("stm_scale_y.npy", scale_y)

print_scale_factors(STEERERS, scale_x, scale_y)

=== Steerer Scale Factors ===
Steerer 1:  s_x=0.8760,   s_y=0.9691
Steerer 2:  s_x=0.9790,   s_y=1.2159
Steerer 3:  s_x=0.7195,   s_y=0.7099
Steerer 4:  s_x=0.8352,   s_y=0.9350
Steerer 5:  s_x=0.7941,   s_y=0.9568


In [8]:
# ---------------------------------------------------------------
# Calibration summary
# ---------------------------------------------------------------
print_calibration_summary(WS, STEERERS, ws_offset_x, ws_offset_y, scale_x, scale_y)

=== SUMMARY ===
Wire Scanner Offsets (apply to simulation or embed into lattice):
  WS1:  dx=-0.2666,  dy=-1.0617
  WS2:  dx=+1.0788,  dy=-0.3785
  WS3:  dx=-0.8454,  dy=-0.5858
  WS4:  dx=-0.2576,  dy=+2.7373

Steerer Scale Factors (multiply TRACK FH/FV or FHkick/FVkick):
  SM1:  x_scale=0.8760,   y_scale=0.9691
  SM2:  x_scale=0.9790,   y_scale=1.2159
  SM3:  x_scale=0.7195,   y_scale=0.7099
  SM4:  x_scale=0.8352,   y_scale=0.9350
  SM5:  x_scale=0.7941,   y_scale=0.9568

Meaning:
- After applying WS offsets, simulation and experiment match at zero kick.
- After applying steerer scale factors, the ±1.1 mrad response matches experiment.

